In [166]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import copy

In [167]:
##FCC Data from June 2019 (Most current)
fcc_bbnd_june19 = pd.read_csv('../data/FCC_Broadband/TN-Fixed-Jun2019-v1.csv')

##For details on original columns: https://www.fcc.gov/general/explanation-broadband-deployment-data
# Code	Technology of Transmission
# 10	Asymmetric xDSL
# 11	ADSL2, ADSL2+
# 12	VDSL
# 20	Symmetric xDSL*
# 30	Other Copper Wireline (all copper-wire based technologies other than xDSL; Ethernet over copper and T-1 are examples)
# 40	Cable Modem other than DOCSIS 1, 1.1, 2.0, 3.0, or 3.1
# 41	Cable Modem – DOCSIS 1, 1.1 or 2.0
# 42	Cable Modem – DOCSIS 3.0
# 43	Cable Modem – DOCSIS 3.1
# 50	Optical Carrier / Fiber to the end user (Fiber to the home or business end user, does not include “fiber to the curb”)
# 60	Satellite
# 70	Terrestrial Fixed Wireless
# 90	Electric Power Line
# 0	All Other

## Urban Area definitions from 2018 Census Geographies 
## 'R' = Rural 'C' = Urban Cluster (2,500-50,000) 'U' = Urbanized Area (50,000+) 
urban_area_shape = gpd.read_file('../data/UrbanAreaShape2018/tl_2018_us_uac10.shp')

## Define central point of TN census tracts from 2010 Census Geographies
centroid_df = pd.read_csv('../data/CenPop2010_Mean_TR47.txt', converters={'TRACTCE': lambda x: str(x), 'COUNTYFP': lambda y: str(y)})

print(fcc_bbnd_june19.shape)
print(fcc_bbnd_june19.info())

print(urban_area_shape.shape)
print(urban_area_shape.info())

print(centroid_df.shape)
print(centroid_df.info())

(1484085, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484085 entries, 0 to 1484084
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   LogRecNo            1484085 non-null  int64  
 1   Provider_Id         1484085 non-null  int64  
 2   FRN                 1484085 non-null  int64  
 3   ProviderName        1484085 non-null  object 
 4   DBAName             1484085 non-null  object 
 5   HoldingCompanyName  1484085 non-null  object 
 6   HocoNum             1484085 non-null  int64  
 7   HocoFinal           1484085 non-null  object 
 8   StateAbbr           1484085 non-null  object 
 9   BlockCode           1484085 non-null  int64  
 10  TechCode            1484085 non-null  int64  
 11  Consumer            1484085 non-null  int64  
 12  MaxAdDown           1484085 non-null  float64
 13  MaxAdUp             1484085 non-null  float64
 14  Business            1484085 non-null  int64  
 15  M

In [168]:
## TN Tracts shape
tract_shape = gpd.read_file('../data/shapefiles/tl_2018_47_tract.shp')


In [169]:
tract_shape.head(50)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,157,007300,47157007300,73,Census Tract 73,G5020,S,1917129,0,+35.1209841,-089.9363799,"POLYGON ((-89.94575 35.11532, -89.94575 35.115..."
1,47,157,021320,47157021320,213.20,Census Tract 213.20,G5020,S,5881320,43891,+35.0925659,-089.8492010,"POLYGON ((-89.87095 35.08952, -89.87077 35.089..."
2,47,157,020222,47157020222,202.22,Census Tract 202.22,G5020,S,34766729,0,+35.2950808,-089.8785477,"POLYGON ((-89.92943 35.26299, -89.92943 35.263..."
3,47,157,022410,47157022410,224.10,Census Tract 224.10,G5020,S,13072034,431366,+35.0111662,-090.0901823,"POLYGON ((-90.12761 34.99468, -90.12732 34.994..."
4,47,157,005300,47157005300,53,Census Tract 53,G5020,S,5967146,497369,+35.0930408,-090.0757059,"POLYGON ((-90.09169 35.08469, -90.09140 35.085..."
5,47,157,008500,47157008500,85,Census Tract 85,G5020,S,6517968,0,+35.1201913,-089.9119212,"POLYGON ((-89.92623 35.10746, -89.92622 35.107..."
6,47,157,020810,47157020810,208.10,Census Tract 208.10,G5020,S,113198234,0,+35.3415264,-089.6995382,"POLYGON ((-89.76606 35.28114, -89.76604 35.281..."
7,47,157,021613,47157021613,216.13,Census Tract 216.13,G5020,S,4563620,34714,+35.0634955,-089.6447390,"POLYGON ((-89.65843 35.04552, -89.65839 35.046..."
8,47,157,021520,47157021520,215.20,Census Tract 215.20,G5020,S,48625004,0,+35.0181428,-089.7116213,"POLYGON ((-89.75992 35.04768, -89.75909 35.047..."
9,47,157,005700,47157005700,57,Census Tract 57,G5020,S,1447788,0,+35.1042256,-090.0410602,"POLYGON ((-90.05336 35.10757, -90.05334 35.107..."


In [170]:
tract_shape.crs
tract_shape.head()
tract_shape = tract_shape[['GEOID', 'geometry']]

In [171]:
tract_shape.shape

(1497, 2)

In [172]:
urban_area_shape = urban_area_shape[['GEOID10', 'NAME10', 'UATYP10', 'ALAND10', 'geometry']]

In [173]:
print(fcc_bbnd_june19.groupby('TechCode').size())
print('\n')
print(fcc_bbnd_june19.groupby('Consumer').size())
print('\n')
print(fcc_bbnd_june19.groupby('Business').size())
print('\n')
print(fcc_bbnd_june19.groupby(['Consumer','Business']).size())

TechCode
10     66864
11    112629
12     67315
20       834
30      9860
40       334
41        61
42     15317
43    121866
50     68334
60    944385
70     76286
dtype: int64


Consumer
0     261604
1    1222481
dtype: int64


Business
0     258113
1    1225972
dtype: int64


Consumer  Business
0         1           261604
1         0           258113
          1           964368
dtype: int64


In [174]:
## Add column for census tract
fcc_bbnd_june19['TractCode'] = fcc_bbnd_june19['BlockCode'].apply(lambda x: str(x)[0:11])

In [175]:
## Drop extraneous columns
all_june19 = fcc_bbnd_june19.drop(['FRN', 'DBAName', 'HoldingCompanyName', 'HocoFinal', 'StateAbbr'], axis=1, inplace=False)

In [176]:
## Add frames with only sat and no sat

only_sat_june19 = fcc_bbnd_june19.loc[fcc_bbnd_june19['TechCode'] == 60]
no_sat_june19 = fcc_bbnd_june19.loc[fcc_bbnd_june19['TechCode'] != 60]

print(all_june19.groupby('TractCode').size())
print(only_sat_june19.groupby('TractCode').size())
print(no_sat_june19.groupby('TractCode').size())
print('\n')
print(all_june19.groupby('BlockCode').size())
print(only_sat_june19.groupby('BlockCode').size())
print(no_sat_june19.groupby('BlockCode').size())
print('\n')
print(all_june19.shape)
print(only_sat_june19.shape)
print(no_sat_june19.shape)

TractCode
47001020100     762
47001020201     509
47001020202     998
47001020300     397
47001020400     765
               ... 
47189030800    2146
47189030901     936
47189030903    1467
47189030904    1659
47189031000    2456
Length: 1497, dtype: int64
TractCode
47001020100     480
47001020201     309
47001020202     614
47001020300     200
47001020400     440
               ... 
47189030800    1512
47189030901     633
47189030903    1076
47189030904    1173
47189031000    1648
Length: 1497, dtype: int64
TractCode
47001020100    282
47001020201    200
47001020202    384
47001020300    197
47001020400    325
              ... 
47189030800    634
47189030901    303
47189030903    391
47189030904    486
47189031000    808
Length: 1497, dtype: int64


BlockCode
470010201001000    5
470010201001001    4
470010201001002    7
470010201001003    6
470010201001004    7
                  ..
471890310004062    8
471890310004063    5
471890310004064    5
471890310004065    7
471890310004066   

In [177]:
## Find how many census blocks are in each tract
count_blocks = no_sat_june19.groupby('TractCode')['BlockCode'].nunique().reset_index()
count_blocks.rename(columns={'BlockCode': 'CountBlocks'}, inplace=True)
count_blocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   TractCode    1497 non-null   object
 1   CountBlocks  1497 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 23.5+ KB


In [178]:
## Merge block count into fcc df
no_sat_june19_countd = no_sat_june19.merge(count_blocks, how='inner', on='TractCode')

In [179]:
## Make dfs which are subsetted for Consumer and Business service, as well as both
consumer_june19 =  no_sat_june19_countd[no_sat_june19_countd['Consumer'] == 1]
business_june19 =  no_sat_june19_countd[no_sat_june19_countd['Business'] == 1]
both_cnsm_biz_june19 = consumer_june19[consumer_june19['Business'] == 1]

consumer_june19_3mbps = consumer_june19[consumer_june19['MaxAdDown'] >= 3.0]
consumer_june19_12mbps = consumer_june19[consumer_june19['MaxAdDown'] >= 12.0]
consumer_june19_25mbps = consumer_june19[consumer_june19['MaxAdDown'] >= 25.0]

business_june19_250mbps = business_june19[business_june19['MaxCIRDown'] >= 250.0]

(463797, 19)
(360695, 19)
(288482, 19)


In [180]:
## Determine how many blocks in a tract have a *choice of services* at levels of any / 3 / 12 / 25 mbps
provider_block_choice = consumer_june19.groupby(['BlockCode', 'TractCode'])['Consumer'].sum().reset_index()
provider_block_choices = provider_block_choice[provider_block_choice['Consumer'] >= 2]

provider_block_choice_3mbps = consumer_june19_3mbps.groupby(['BlockCode', 'TractCode'])['Consumer'].sum().reset_index()
provider_block_choices_3mbps = provider_block_choice_3mbps[provider_block_choice_3mbps['Consumer'] >= 2]

provider_block_choice_12mbps = consumer_june19_12mbps.groupby(['BlockCode', 'TractCode'])['Consumer'].sum().reset_index()
provider_block_choices_12mbps = provider_block_choice_12mbps[provider_block_choice_12mbps['Consumer'] >= 2]

provider_block_choice_25mbps = consumer_june19_25mbps.groupby(['BlockCode', 'TractCode'])['Consumer'].sum().reset_index()
provider_block_choices_25mbps = provider_block_choice_25mbps[provider_block_choice_25mbps['Consumer'] >= 2]

block_choices_counted = provider_block_choices.groupby(['TractCode']).count().reset_index()
block_choices_counted.drop('BlockCode', axis=1, inplace=True)
block_choices_counted.rename(columns={'Consumer': 'Blocks With Choice'}, inplace=True)

block_choices_counted_3mbps = provider_block_choices_3mbps.groupby(['TractCode']).count().reset_index()
block_choices_counted_3mbps.drop('BlockCode', axis=1, inplace=True)
block_choices_counted_3mbps.rename(columns={'Consumer': 'Blocks With Choice 3mbps'}, inplace=True)

block_choices_counted_12mbps = provider_block_choices_12mbps.groupby(['TractCode']).count().reset_index()
block_choices_counted_12mbps.drop('BlockCode', axis=1, inplace=True)
block_choices_counted_12mbps.rename(columns={'Consumer': 'Blocks With Choice 12mbps'}, inplace=True)

block_choices_counted_25mbps = provider_block_choices_25mbps.groupby(['TractCode']).count().reset_index()
block_choices_counted_25mbps.drop('BlockCode', axis=1, inplace=True)
block_choices_counted_25mbps.rename(columns={'Consumer': 'Blocks With Choice 25mbps'}, inplace=True)

consumer_choice_dfs = [block_choices_counted, block_choices_counted_3mbps, block_choices_counted_12mbps, block_choices_counted_25mbps]

In [181]:
## Create dataframe to show proportion of blocks in each tract with a choice
choice_levels_add = count_blocks.copy()
for df in consumer_choice_dfs:    
    choice_levels_add = choice_levels_add.merge(df, how='left', on='TractCode')
    
choice_levels_add['Prop With Choice'] = round((100 * choice_levels_add['Blocks With Choice'] / choice_levels_add['CountBlocks']), 2)
choice_levels_add['Prop With Choice 3mbps'] = round((100 * choice_levels_add['Blocks With Choice 3mbps'] / choice_levels_add['CountBlocks']), 2)
choice_levels_add['Prop With Choice 12mbps'] = round((100 * choice_levels_add['Blocks With Choice 12mbps'] / choice_levels_add['CountBlocks']), 2)
choice_levels_add['Prop With Choice 25mbps'] = round((100 * choice_levels_add['Blocks With Choice 25mbps'] / choice_levels_add['CountBlocks']), 2)

In [182]:
choice_levels_add = choice_levels_add.fillna(0)
choice_levels_add.to_csv('../out/choice_levels.csv')

In [183]:
## Determine how many blocks in a tract are served *by a given provider*.
provider_block_served_consumer = consumer_june19.groupby(['ProviderName', 'BlockCode', 'TractCode'])['Consumer'].max().reset_index()
provider_block_served_business = business_june19.groupby(['ProviderName', 'BlockCode', 'TractCode'])['Business'].max().reset_index()

sum_blocks_by_tract_consumer = provider_block_served_consumer.groupby(['ProviderName','TractCode'])['Consumer'].sum().reset_index()
sum_blocks_by_tract_business = provider_block_served_business.groupby(['ProviderName','TractCode'])['Business'].sum().reset_index()

sum_blocks_by_tract_consumer.rename(columns={'Consumer': 'Consumer Blocks Served'}, inplace=True)
sum_blocks_by_tract_business.rename(columns={'Business': 'Business Blocks Served'}, inplace=True)

In [184]:
## Determine how many provider blocks in a tract are served at levels of 3 / 12 / 25 mbps as per this: https://www.fcc.gov/consumers/guides/household-broadband-guide
provider_block_served_consumer_3mbps = consumer_june19_3mbps.groupby(['ProviderName', 'BlockCode', 'TractCode'])['Consumer'].max().reset_index()
provider_block_served_consumer_12mbps = consumer_june19_12mbps.groupby(['ProviderName', 'BlockCode', 'TractCode'])['Consumer'].max().reset_index()
provider_block_served_consumer_25mbps = consumer_june19_25mbps.groupby(['ProviderName', 'BlockCode', 'TractCode'])['Consumer'].max().reset_index()

provider_block_served_business_250mbps = business_june19_250mbps.groupby(['ProviderName', 'BlockCode', 'TractCode'])['Business'].max().reset_index()

sum_blocks_by_tract_consumer_3mbps = provider_block_served_consumer_3mbps.groupby(['ProviderName','TractCode'])['Consumer'].sum().reset_index()
sum_blocks_by_tract_consumer_12mbps = provider_block_served_consumer_12mbps.groupby(['ProviderName','TractCode'])['Consumer'].sum().reset_index()
sum_blocks_by_tract_consumer_25mbps = provider_block_served_consumer_25mbps.groupby(['ProviderName','TractCode'])['Consumer'].sum().reset_index()

sum_blocks_by_tract_business_250mbps = provider_block_served_business_250mbps.groupby(['ProviderName','TractCode'])['Business'].sum().reset_index()

sum_blocks_by_tract_consumer_3mbps.rename(columns={'Consumer': 'Consumer Blocks Served 3mbps'}, inplace=True)
sum_blocks_by_tract_consumer_12mbps.rename(columns={'Consumer': 'Consumer Blocks Served 12mbps'}, inplace=True)
sum_blocks_by_tract_consumer_25mbps.rename(columns={'Consumer': 'Consumer Blocks Served 25mbps'}, inplace=True)

sum_blocks_by_tract_business_250mbps.rename(columns={'Business': 'Business Blocks Served 250mbps'}, inplace=True)

In [185]:
## Determine how many blocks in a tract are served *by anyone*.
block_served_consumer = consumer_june19.groupby(['BlockCode', 'TractCode'])['Consumer'].max().reset_index()
block_served_business = business_june19.groupby(['BlockCode', 'TractCode'])['Business'].max().reset_index()

blocks_served_any_pro_consumer = block_served_consumer.groupby(['TractCode'])['Consumer'].sum().reset_index()
blocks_served_any_pro_business = block_served_business.groupby(['TractCode'])['Business'].sum().reset_index()

blocks_served_any_pro_consumer.rename(columns={'Consumer': 'Consumer Blocks Served Any Pro'}, inplace=True)
blocks_served_any_pro_business.rename(columns={'Business': 'Business Blocks Served Any Pro'}, inplace=True)

In [187]:
## Determine how many blocks in a tract are served at levels of 3 / 12 / 25 mbps by any provider as per this: https://www.fcc.gov/consumers/guides/household-broadband-guide
block_served_consumer_3mbps = consumer_june19_3mbps.groupby(['BlockCode', 'TractCode'])['Consumer'].max().reset_index()
block_served_consumer_12mbps = consumer_june19_12mbps.groupby(['BlockCode', 'TractCode'])['Consumer'].max().reset_index()
block_served_consumer_25mbps = consumer_june19_25mbps.groupby(['BlockCode', 'TractCode'])['Consumer'].max().reset_index()

block_served_business_250mbps = business_june19_250mbps.groupby(['BlockCode', 'TractCode'])['Business'].max().reset_index()

blocks_served_any_pro_consumer_3mbps = block_served_consumer_3mbps.groupby(['TractCode'])['Consumer'].sum().reset_index()
blocks_served_any_pro_consumer_12mbps = block_served_consumer_12mbps.groupby(['TractCode'])['Consumer'].sum().reset_index()
blocks_served_any_pro_consumer_25mbps = block_served_consumer_25mbps.groupby(['TractCode'])['Consumer'].sum().reset_index()

blocks_served_any_pro_business_250mbps = block_served_business_250mbps.groupby(['TractCode'])['Business'].sum().reset_index()

blocks_served_any_pro_consumer_3mbps.rename(columns={'Consumer': 'Consumer Blocks Served Any Pro 3mbps'}, inplace=True)
blocks_served_any_pro_consumer_12mbps.rename(columns={'Consumer': 'Consumer Blocks Served Any Pro 12mbps'}, inplace=True)
blocks_served_any_pro_consumer_25mbps.rename(columns={'Consumer': 'Consumer Blocks Served Any Pro 25mbps'}, inplace=True)

blocks_served_any_pro_business_250mbps.rename(columns={'Business': 'Business Blocks Served Any Pro 250mbps'}, inplace=True)


In [189]:
consumer_count_any_pro = count_blocks.copy()
consumer_any_pro_dfs = [blocks_served_any_pro_consumer, blocks_served_any_pro_consumer_3mbps, blocks_served_any_pro_consumer_12mbps, blocks_served_any_pro_consumer_25mbps]
for df in consumer_any_pro_dfs:    
    consumer_count_any_pro = consumer_count_any_pro.merge(df, how='left', on='TractCode')

consumer_count_any_pro['Prop Served By Any'] = round((100 * consumer_count_any_pro['Consumer Blocks Served Any Pro'] / consumer_count_any_pro['CountBlocks']), 2)
consumer_count_any_pro['Prop Served By Any 3mbps'] = round((100 * consumer_count_any_pro['Consumer Blocks Served Any Pro 3mbps'] / consumer_count_any_pro['CountBlocks']), 2)
consumer_count_any_pro['Prop Served By Any 12mbps'] = round((100 * consumer_count_any_pro['Consumer Blocks Served Any Pro 12mbps'] / consumer_count_any_pro['CountBlocks']), 2)
consumer_count_any_pro['Prop Served By Any 25mbps'] = round((100 * consumer_count_any_pro['Consumer Blocks Served Any Pro 25mbps'] / consumer_count_any_pro['CountBlocks']), 2)

In [190]:
consumer_count_any_pro = consumer_count_any_pro.fillna(0)
consumer_count_any_pro.to_csv('../out/consumers_any_provider.csv')

In [125]:
short_consumer_june19 = consumer_june19[['ProviderName', 'TechCode', 'TractCode', 'CountBlocks']]

short_consumer_combo = short_consumer_june19.merge(sum_blocks_by_tract_consumer, how='left', on=['ProviderName', 'TractCode'])
short_consumer_combo = short_consumer_combo.merge(sum_blocks_by_tract_consumer_3mbps, how='left', on=['ProviderName', 'TractCode'])
short_consumer_combo = short_consumer_combo.merge(sum_blocks_by_tract_consumer_12mbps, how='left', on=['ProviderName', 'TractCode'])
short_consumer_combo = short_consumer_combo.merge(sum_blocks_by_tract_consumer_25mbps, how='left', on=['ProviderName', 'TractCode'])

short_consumer_combo.drop_duplicates(inplace=True)
short_consumer_combo = short_consumer_combo.fillna(0)
short_consumer_combo = short_consumer_combo.astype({'Consumer Blocks Served 3mbps': 'int64', 
                                                    'Consumer Blocks Served 12mbps': 'int64',
                                                    'Consumer Blocks Served 25mbps': 'int64'}, copy=False)
short_consumer_combo.info()
##short_consumer_combo.to_csv('../out/consumer_reduced.csv')
##def combine_frame_on_tract

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9436 entries, 0 to 512740
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ProviderName                     9436 non-null   object
 1   TechCode                         9436 non-null   int64 
 2   TractCode                        9436 non-null   object
 3   CountBlocks                      9436 non-null   int64 
 4   Consumer Blocks Served           9436 non-null   int64 
 5   Consumer Blocks Served - 3mbps   9436 non-null   int64 
 6   Consumer Blocks Served - 12mbps  9436 non-null   int64 
 7   Consumer Blocks Served - 25mbps  9436 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 663.5+ KB


In [126]:
## In each tract that is served by a provider, how many blocks do they serve?
## Consumer:

##block_sum_consumers = consumer_june19.groupby(['TractCode', 'ProviderName', 'CountBlocks', 'TechCode'])['Consumer'].sum().reset_index()
## Business
##block_sum_businesses = business_june19.groupby(['TractCode', 'ProviderName', 'CountBlocks', 'TechCode'])['Business'].sum().reset_index()

short_consumer_combo['Proportion Served'] = round((100 * short_consumer_combo['Consumer Blocks Served'] / short_consumer_combo['CountBlocks']), 2)
short_consumer_combo['Proportion Served 3mbps'] = round((100 * short_consumer_combo['Consumer Blocks Served 3mbps'] / short_consumer_combo['CountBlocks']), 2)
short_consumer_combo['Proportion Served 12mbps'] = round((100 * short_consumer_combo['Consumer Blocks Served 12mbps'] / short_consumer_combo['CountBlocks']), 2)
short_consumer_combo['Proportion Served 25mbps'] = round((100 * short_consumer_combo['Consumer Blocks Served 25mbps'] / short_consumer_combo['CountBlocks']), 2)

short_consumer_combo['Substantially Served'] = short_consumer_combo['Proportion Served'] >= 30.0
short_consumer_combo['Substantially Served 3mbps'] = short_consumer_combo['Proportion Served 3mbps'] >= 30.0
short_consumer_combo['Substantially Served 12mbps'] = short_consumer_combo['Proportion Served 12mbps'] >= 30.0
short_consumer_combo['Substantially Served 25mbps'] = short_consumer_combo['Proportion Served 25mbps'] >= 30.0

In [127]:
short_consumer_for_set = short_consumer_combo.copy()

In [128]:
consumer_combo_no_code = short_consumer_combo.drop('TechCode', axis=1, inplace=False)
consumer_combo_no_code.drop_duplicates(inplace=True)
set_codes = short_consumer_for_set.groupby(['ProviderName', 'TractCode'])['TechCode'].agg(lambda x : set(x)).reset_index()
consumer_set_codes = consumer_combo_no_code.merge(set_codes, how='left', on=['ProviderName', 'TractCode'])

In [129]:
consumer_set_codes['Minimally Residential'] = consumer_set_codes['TractCode'].map(lambda x: x[5:7] == '98')
consumer_set_codes['Water Area'] = consumer_set_codes['TractCode'].map(lambda x: x[5:7] == '99')
consumer_set_codes['Consumer Service N/A'] = consumer_set_codes['Water Area'] ^ consumer_set_codes['Minimally Residential']

In [130]:
## TODO: Expand the set to a column for each
consumer_set_codes.rename({'TechCode': 'TechCodes Set'}, axis=1, inplace=True)
consumer_set_codes.to_csv('../out/consumers_served.csv')

In [132]:
providers_serving_tract = consumer_set_codes.groupby('TractCode')[['Substantially Served', 'Substantially Served 3mbps', 'Substantially Served 12mbps', 'Substantially Served 25mbps']].sum().reset_index()
providers_serving_tract.to_csv('../out/providers_serving_tract.csv')

In [ ]:
census_url = 'https://api.census.gov/data'
year = '2018'
dataset = 'acs/acs5'
base_url = '/'.join([census_url, year, dataset])
get_tract_vars = ['B01003_001E']
get_tract_vars = ["NAME"] + get_tract_vars
predicates = {}
predicates['get'] = ','.join(get_tract_vars)
predicates['for'] = 'tract:*'
predicates['in'] = 'state:47'
request = requests.get(base_url, params=predicates)
col_names = ['NAME', 'Population', 'state', 'county', 'tract']
census_tract_df = pd.DataFrame(data=request.json()[0:], columns=col_names)

In [ ]:
census_tract_df['TractCode'] = census_tract_df['state'] + census_tract_df['county'] + census_tract_df['tract']
census_tract_two_col = census_tract_df[['Population', 'TractCode']]

In [77]:
mappable_consumers = tract_shape.merge(consumer_set_codes, how='left', left_on='GEOID', right_on='TractCode')
mappable_provider_tract = tract_shape.merge(providers_serving_tract, how='left', left_on='GEOID', right_on='TractCode')
mappable_consumers.to_csv('../out/mappable_consumer_with_providers.csv')
mappable_provider_tract.to_csv('../out/mappable_providers_serving_tract.csv')

In [ ]:
map_top_providers_dfs = []
top_n_providers = list(block_sum_consumers.groupby('ProviderName').size().sort_values().tail(10).index)
rp = copy.copy(cm.get_cmap('tab20c'))
for x in top_n_providers:
    provider_sub = block_sum_consumers[block_sum_consumers['ProviderName'] == x]
    mappable = tract_shape.merge(provider_sub, how='left', left_on='GEOID', right_on='TractCode')
    print(mappable.shape)
    map_top_providers_dfs.append(mappable)
for provider_map in map_top_providers_dfs:
    provider_map.plot(column='Substantially Served', cmap=rp)

In [ ]:
charter_sub = block_sum_consumers[block_sum_consumers['ProviderName'] == 'Charter Communications, Inc.']

In [ ]:
charter_geo = tract_shape.merge(charter_sub, how='outer', left_on='GEOID', right_on='TractCode')

In [ ]:
charter_geo_filled = charter_geo.fillna(0.0001)

In [ ]:
charter_geo_filled.plot(column='Substantially Served')

In [ ]:
sub_tract = no_sat_june19[no_sat_june19['TractCode'] == '47189031000']
sub_tract.head(20)
sub_tract.groupby(['BlockCode']).size()

In [ ]:
centroid_df.drop('POPULATION', axis=1, inplace=True)

In [ ]:
centroid_df = centroid_df.astype({'STATEFP': 'str'}, copy=False)

In [ ]:
centroid_df['TractCode'] = centroid_df['STATEFP'] + centroid_df['COUNTYFP'] + centroid_df['TRACTCE']

In [ ]:
##Compare centroid tracts to fcc data tracts - to show there are no tracts without data in the fcc no satellite group
no_sat_tract = no_sat_june19['TractCode'].unique()
centroid_tract = centroid_df['TractCode']
print(no_sat_tract.shape)
print(centroid_tract.shape)

In [ ]:
centroid_df['geometry'] = centroid_df.apply(lambda x: Point((float(x['LONGITUDE']), float(x['LATITUDE']))), axis=1)

In [ ]:
centroid_geo = gpd.GeoDataFrame(centroid_df, crs = urban_area_shape.crs, geometry = centroid_df['geometry'])
centroid_geo = centroid_geo[['LATITUDE', 'LONGITUDE', 'TractCode', 'geometry']]

In [ ]:
centroid_geo.shape

In [ ]:
ua_of_census_tracts = gpd.sjoin(centroid_geo, urban_area_shape, op = 'within')
ua_of_census_tracts.head()

In [ ]:
ua_of_census_tracts.drop('index_right', axis=1, inplace=True)

In [ ]:
##Find any rows where Tract is not in the spatially merged df
centroid_geo_rural = centroid_geo[centroid_geo['TractCode'].isin(ua_of_census_tracts['TractCode'].tolist()) == False]
centroid_geo_rural.info()

In [ ]:
centroid_rural_match_cols = pd.concat([centroid_geo_rural, pd.DataFrame(columns=['GEOID10', 'NAME10', 'UATYP10', 'ALAND10'])])
centroid_rural_match_cols['UATYP10'] = 'R'
centroid_rural_match_cols['ALAND10'] = centroid_rural_match_cols['ALAND10'].fillna(np.nan)

In [ ]:
centroid_rural_match_cols.head()

In [ ]:
tract_by_ua_w_rural = pd.concat([centroid_rural_match_cols, ua_of_census_tracts], axis=0)

In [ ]:
print(tract_by_ua_w_rural.shape)
print(tract_by_ua_w_rural.info())
tract_by_ua_w_rural.head(50)
tract_by_ua_w_rural['UATYP10'].unique()

In [ ]:
census_url = 'https://api.census.gov/data'
year = '2010'
dataset = 'dec/sf1'
base_url = '/'.join([census_url, year, dataset])
get_tract_vars = ['P001001']
get_tract_vars = ["NAME"] + get_tract_vars
predicates = {}
predicates['get'] = ','.join(get_tract_vars)
predicates['for'] = 'urban area:*'  ## zctas
##predicates['in'] = 'state:47'
request = requests.get(base_url, params=predicates)
##col_names = ['NAME', 'Population', 'zip']
census_df = pd.DataFrame(data=request.json()[1:])

##https://api.census.gov/data/2010/dec/sf1?get=P001001,NAME&for=urban%20rural:*&in=state:01%20county:087%20county%20subdivision:93230%20place/remainder%20(or%20part):77304%20tract%20(or%20part):231900

In [ ]:
census_url = 'https://api.census.gov/data'
year = '2018'
dataset = 'acs/acs5'
base_url = '/'.join([census_url, year, dataset])
get_tract_vars = ['B01003_001E']
get_tract_vars = ["NAME"] + get_tract_vars
predicates = {}
predicates['get'] = ','.join(get_tract_vars)
predicates['for'] = 'urban area:*'  ## zctas
##predicates['in'] = 'urban area:*'

request = requests.get(base_url, params=predicates)
##col_names = ['NAME', 'Population', 'zip']
census_df = pd.DataFrame(data=request.json()[1:])

In [ ]:
census_df.head()

,NAME,Population,state,county,tract
25,"Census Tract 9801, Cocke County, Tennessee",0,47,029,980100
539,"Census Tract 9801, Shelby County, Tennessee",76,47,157,980100
544,"Census Tract 9801, Stewart County, Tennessee",0,47,161,980100
736,"Census Tract 9801, Anderson County, Tennessee",0,47,001,980100
757,"Census Tract 9801, Sevier County, Tennessee",0,47,155,980100
798,"Census Tract 9801, Blount County, Tennessee",5,47,009,980100
986,"Census Tract 9801, Carroll County, Tennessee",0,47,017,980100
1087,"Census Tract 9801, Coffee County, Tennessee",4,47,031,980100
1124,"Census Tract 9801, Hamilton County, Tennessee",0,47,065,980100
1409,"Census Tract 9801, Davidson County, Tennessee",0,47,037,980100
